In [1]:
import os
import pickle
from catboost import CatBoostClassifier
import pandas as pd
from sqlalchemy import create_engine

def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":  # проверяем где выполняется код в лмс, или локально. Немного магии
        MODEL_PATH = '/workdir/user_input/model'
    else:
        MODEL_PATH = path
    return MODEL_PATH

def load_models():
    from_file = CatBoostClassifier()
    # LOAD MODEL HERE PLS :)
    loaded_model = from_file.load_model(get_model_path("catboost_model"))
    return loaded_model

def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [2]:
%%time
from sklearn.decomposition import PCA 
user_data_limit = 163205 #114903
user_data = batch_load_sql(f"SELECT * FROM public.user_data LIMIT {user_data_limit}")
user_data

CPU times: total: 969 ms
Wall time: 19.4 s


,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [3]:
#test
user_data[user_data['user_id'] == 2000000].shape

(0, 8)

In [4]:
%%time
post_text_limit = 1000 #6924
post_text = batch_load_sql(f"SELECT * FROM public.post_text_df LIMIT {post_text_limit}")
post_text = post_text.iloc[post_text.drop('post_id', axis=1).drop_duplicates().index.tolist()]
post_text

CPU times: total: 15.6 ms
Wall time: 4.59 s


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
995,2535,Excellent quality cloth masks are available at...,covid
996,2536,🚨 UPDATE 🚨 out of #NewJersey from @GovMurphy! ...,covid
997,2537,Heartbreaking piece on coronavirus in the Amaz...,covid
998,2538,Exactly how long can this bloody #covid19 hide...,covid


In [5]:
#%%time
#feed_data_limit = 500000
#feed_data = batch_load_sql(f"SELECT * FROM public.feed_data LIMIT {feed_data_limit}")

In [6]:
test_df = pd.concat([user_data, post_text])
test_df

,user_id,gender,age,country,city,exp_group,os,source,post_id,text,topic
0,200.0,1.0,34.0,Russia,Degtyarsk,3.0,Android,ads,NaN,NaN,NaN
1,201.0,0.0,37.0,Russia,Abakan,0.0,Android,ads,NaN,NaN,NaN
2,202.0,1.0,17.0,Russia,Smolensk,4.0,Android,ads,NaN,NaN,NaN
3,203.0,0.0,18.0,Russia,Moscow,1.0,iOS,ads,NaN,NaN,NaN
4,204.0,0.0,36.0,Russia,Anzhero-Sudzhensk,3.0,Android,ads,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2535.0,Excellent quality cloth masks are available at...,covid
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2536.0,🚨 UPDATE 🚨 out of #NewJersey from @GovMurphy! ...,covid
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2537.0,Heartbreaking piece on coronavirus in the Amaz...,covid
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2538.0,Exactly how long can this bloody #covid19 hide...,covid


In [7]:
%%time
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

test_df.to_sql('denis21.97@mail.ru_lesson_22', con=engine, if_exists="replace", index=False, method='multi') 
# записываем таблицу #chunksize=200000

CPU times: total: 29.9 s
Wall time: 1min 10s
